# Chapter 05 오차역전파법

__수치 미분__
- 단순하고 쉽다
- 계산 시간이 오래 걸린다

$\rightarrow$ __오차역전파법(backpropagation)__

## 5.1 계산 그래프
P.148

__게산 그래프__ : 계산 과정을 그래프로 나타낸 것

### 5.1.1 계산 그래프로 풀다

<img src='./img/fig 5-1.png' width=500>

<img src='./img/fig 5-2.png' width=500>

<img src='./img/fig 5-3.png' width=500>

__순전파__ : 계산을 왼쪽에서 오른쪽으로 진행

__역전파__ : 계산을 오른쪽에서 왼쪽으로 진행

### 5.1.2 국소적 계산

국소적 계산 : 자신과 직접 관련된 작은 범위
>전체와 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다.

<img src='./img/fig 5-4.png' width=500>

### 5.1.3 왜 계산 그래프로 푸는가?

1. 국소적 계산
2. 중간 계산 결과를 모두 보관
3. 역전파를 통해 __미분__을 효과적으로 계산

<img src='./img/fig 5-5.png' width=500>

## 5.2 연쇄법칙
P.152

### 5.2.1 계산 그래프의 역전파

신호 E에 노드의 국소적 미분($\frac{\partial y}{\partial x}$)을 곱한 후 다음 노드로 전달

<img src='./img/fig 5-6.png' width=250>

### 5.2.2 연쇄법칙이란?

_**합성 함수**의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다._

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$$

### 5.2.3 연쇄법칙과 계산 그래프

<img src='./img/fig 5-7.png' width=400>

노드로 들어온 입력 신호에 그 노드의 국소적 미분(편미분)을 곱한 후 다음 노드로 전달


<img src='./img/fig 5-8.png' width=400>

## 5.3 역전파
P.155

### 5.3.1 덧셈 노드의 역전파

$$z = x + y$$

$$\frac{\partial z}{\partial x} = 1$$

$$\frac{\partial z}{\partial y} = 1$$

>덧셈 노드의 역전파는 입력된 값을 그대로 다음 노드로 보냄
>
>순방향 입력 신호의 값이 필요 없음

<img src='./img/fig 5-9.png' width=500>

<img src='./img/fig 5-10.png' width=500>


예시)
<img src='./img/fig 5-11.png' width=500>

### 5.3.2 곱셈 노드의 역전파

$$z = xy$$

$$\frac{\partial z}{\partial x} = y$$

$$\frac{\partial z}{\partial y} = x$$

>곱셈 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보냄
>
>순방향 입력 신호의 값이 필요함 $\Rightarrow$ 순전파의 입력 신호를 변수에 저장

<img src='./img/fig 5-12.png' width=500>


예시)
<img src='./img/fig 5-13.png' width=500>

### 5.3.3 사과 쇼핑의 예

<img src='./img/fig 5-14.png' width=500>

<img src='./img/fig 5-17.png' width=500>

## 5.4 단순한 계층 구현하기
P.160

신경망을 구성하는 '계층' 각각을 하나의 클래스로 구현

__계층__ : 신경망의 기능 단위.  e.g. Sigmoid, Affine

### 5.4.1 곱셈 계층

In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

<img src='./img/fig 5-16.png' width=500>

In [9]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


### 5.4.2 덧셈 계층

In [10]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

<img src='./img/fig 5-17.png' width=500>

In [12]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기
P.165

### 5.5.1 ReLU 계층

$$y = 
\begin{cases}
x \quad (x > 0)\\
0 \quad (x \leq 0)
\end{cases}$$

$$\frac{\partial y}{\partial x} = 
\begin{cases}
1 \quad (x > 0)\\
0 \quad (x \leq 0)
\end{cases}$$

- 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보냄.
- 순전파 때의 입력인 x가 0 이하면 역전파는 하류로 신호를 보내지 않음.


<img src='./img/fig 5-18.png' width=600>

In [13]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)  # mask는 True/False로 구성된 넘파이 배열
        out = x.copy()
        out[self.mask] = 0  # x의 원소 값이 0 이하인 원소는 0으로, 나머지는 그대로
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0  # x의 원소의 값이 0 이하면 역전파 때의 값이 0, 나머지는 그대로
        dx = dout
        return dout

In [16]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### 5.5.2 Sigmoid 계층

$$y = \frac{1}{1 + exp(-x)}$$

<img src='./img/fig 5-19.png' width=600>

**1 단계**

$$\frac{\partial y}{\partial x} = - \frac{1}{x^2} = - y^2$$

<img src='./img/fig 5-19(1).png' width=600>

**2 단계**
<img src='./img/fig 5-19(2).png' width=600>

**3 단계**

$$\frac{\partial y}{\partial x} = exp(x)$$

<img src='./img/fig 5-19(3).png' width=600>

**4 단계**
<img src='./img/fig 5-20.png' width=600>

**간소화 버전**

- 순전파의 입력 x와 출력 y만으로 계산할 수 있음.
<img src='./img/fig 5-21.png' width=300>


- 순전파의 출력 y만으로 계산할 수 있음
<img src='./img/e 5.12.png' width=300>

<img src='./img/fig 5-22.png' width=300>

In [17]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 5.6 Affine/Softmax 계층 구현하기
P.170

### 5.6.1 Affine 계층

행렬의 곱

In [19]:
X = np.random.rand(2)  # 입력
W = np.random.rand(2,3)  # 가중치
B = np.random.rand(3)  # 편향

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


<img src='./img/fig 5-24.png' width=300>

$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot W^T$$

$$\frac{\partial L}{\partial W} = X^T \cdot \frac{\partial L}{\partial Y}$$

- $W^T$ : 전치행렬. (i, j) 위치의 원소를 (j, i) 위치로 바꾼 것

<img src='./img/e 5.14.png' width=160>

<img src='./img/fig 5-25.png' width=500>

### 5.6.2 배치용 Affine 계층

<img src='./img/fig 5-27.png' width=500>

- 편향의 역전파는 데이터에 대한 미분을 데이터마다 더해서 구한다.

In [20]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [22]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [23]:
class Affine:
    def __init__(self):
        self.W = None
        self.b = None
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### 5.6.3 Softmax-with-Loss 계층

<img src='./img/fig 5-28.png' width=600>
Softmax 계층은 입력 값을 정규화(출력의 합이 1)하여 출력함.

※ NOTE_
- 추론 - Softmax 계층을 사용하지 않음
- 학습 - Softmax 계층이 필요함

<img src='./img/fig 5-29.png' width=600>

<img src='./img/fig 5-30.png' width=550>

($y_1 - t_1, \, y_2 - t_2, \, y_3 - t_3$)

역전파의 결과가 Softmax 계층의 출력과 정답 레이블의 차분, 즉 __현재 출력과 정답 레이블의 오차__

※ NOTE_

우연이 아니라 그렇게 설계되었기 때문
- 소프트맥스 함수 - 교차 엔트로피 오차
- 항등 함수 - 오차제곱합

In [24]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

## 5.7 오차역전파법 구현하기
P.179

### 5.7.1 신경망 학습의 전체 그림

**전제**<br>
신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 합니다. <br>
신경망 학습은 다음과 같이 4단계로 수행합니다.

**1단계 - 미니배치**<br>
훈련 데이터 중 일부를 무작위로 가져옵니다. <br>
이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 중이는 것이 목표입니다.

**2단계 - 기울기 산출**<br>
미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다. <br>
기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시합니다.

**3단계 - 매개변수 갱신**<br>
가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다.

**4단계 - 반복**<br>
1~3단계를 반복합니다.

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [29]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):  # x : 입력 데이터, t : 정답 레이블
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim !=1:
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):  # x : 입력 데이터, t : 정답 레이블
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

In [27]:
import numpy as np
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ':' + str(diff))

W1:4.209424773209219e-10
b1:2.3861360132687365e-09
W2:5.960658212013435e-09
b2:1.4045622585279194e-07


### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [30]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.08633333333333333 0.0894
0.9031333333333333 0.9081
0.9236833333333333 0.9293
0.9371 0.9355
0.9458166666666666 0.9438
0.9497166666666667 0.9464
0.9571 0.952
0.9605666666666667 0.9559
0.9636 0.9579
0.9654166666666667 0.9597
0.9705 0.9644
0.97225 0.9644
0.97355 0.9675
0.9763 0.967
0.9778833333333333 0.9688
0.9780666666666666 0.9696
0.9794333333333334 0.9705
